# Support Vector Machine

### Data Loading

In [10]:
import pandas as pd
import numpy as np

trn_imputed_dum_X = pd.read_csv('./Dataset/trn_imputed_dum_X.csv')
tst_imputed_dum_X = pd.read_csv('./Dataset/tst_imputed_dum_X.csv')
trn_origin_Y      = pd.read_csv('./Dataset/trn_origin_Y.csv')
tst_origin_X      = pd.read_csv('./Dataset/tst_origin_X.csv')

print(trn_imputed_dum_X.shape)
print(trn_origin_Y.shape)

(79853, 17)
(79853, 1)


### 1) Hyperparameter optimization

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

param = {'C':[0.001, 0.01, 0.05, 0.1, 0.5, 1]}

def clf_SVC(trn_X, trn_Y, svc_param):
    clf_SVC = SVC(probability=True)
    clf = GridSearchCV(clf_SVC, svc_param, n_jobs=8, cv=10, scoring='roc_auc')
    clf.fit(trn_X, trn_Y)
    return clf  

### 2) Training Model

In [12]:
clf_model = clf_SVC(trn_imputed_dum_X, trn_origin_Y.values.ravel(), param)

KeyboardInterrupt: 

In [5]:
clf_model.grid_scores_

/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.55111, std: 0.01133, params: {'C': 0.001},
 mean: 0.66159, std: 0.00407, params: {'C': 0.01},
 mean: 0.66760, std: 0.00327, params: {'C': 0.1},
 mean: 0.66752, std: 0.00321, params: {'C': 1},
 mean: 0.66137, std: 0.00363, params: {'C': 10}]


### 3) DO PREDICTION

In [6]:
clf_yhat = clf_model.best_estimator_.predict(tst_imputed_dum_X)
clf_prob = clf_model.best_estimator_.predict_proba(tst_imputed_dum_X)
clf_prob_renewal = clf_prob[:,1]
display(clf_prob_renewal)

AttributeError: predict_proba is not available when  probability=False

### 4) Export to output file

In [ ]:
tst_output = pd.DataFrame(tst_origin_X[['id', 'premium']])

clf_prob_df = pd.DataFrame(clf_prob_renewal.reshape(len(clf_prob_renewal),1), columns=['renewal'])
pd_incentives = pd.DataFrame(np.zeros((len(clf_prob_renewal), 1)), columns=['incentives'], dtype=np.float32)
pd_improve = pd.DataFrame(np.zeros((len(clf_prob_renewal), 1)), columns=['improvement'], dtype=np.float32)

tst_output = pd.concat([tst_out, clf_prob_df, pd_improve, pd_incentives], axis=1)
#display(tst_output)

tst_output.to_csv('./Dataset/tst_renewal_predicted_SVM.csv', index=False)